In [8]:
# import lib
import numpy as np 
import pandas as pd 
import os
import shutil
from os import listdir
from os.path import isfile, join

In [9]:
# folder/ dir/ path
path = 'C:\\Users\\Administrator\\Desktop\\LSF\\RPA-Python\\csv\\PVcheque\\'
# dir = os.path.dirname(os.path.realpath(__file__))
wwwpath = 'C:\\inetpub\\wwwroot\\report\\PVreport\\PV1\\'
filesname = [f for f in listdir(path) if isfile(join(path, f))]
latestfile = sorted(filesname,reverse=True)[0]

In [10]:
# opencsv 
df = pd.read_csv(path+latestfile)

new_col = ['จ่ายให้','รายละเอียด','จำนวนเงินรวมทั้งสิ้น','เลขที่ cheque/ประเภทการจ่าย','ลงวันที่จ่าย','A/C No.','BANK','AP NO.','status']

# df.columns

In [11]:
# df
def waitpay(string):
    try:
        if ("Waiting Payment" in string): return True
        else: return False
    except:
        return False    

In [12]:
# recrate col
def formatDate(stringDate):
    try:
        D = stringDate.split("/")
        d = D[1]; m = D[0]; yr = D[2]
        return d+'/'+m+'/'+yr
    except:
        return 'NaN'  

def isNotBlank (myString):
    if myString and myString.strip():
        #myString is not None AND myString is not empty or blank
        return True
    #myString is None OR myString is empty or blank
    return False

def isBlank (myString):
    if myString and myString.strip():
        #myString is not None AND myString is not empty or blank
        return False
    #myString is None OR myString is empty or blank
    return True



def formatAP(stringTask):
    try:
        P = stringTask.split("|")
        pvtype = P[0]; pvno = P[1]; AP = P[2]; haspaid=P[3]
        if(isNotBlank(AP)): return AP.strip()
        else : return 'NaN'
    except:
        return 'NaN' 

def fillPaid(s):
    if(isNotBlank(str(s)) and str(s) != "nan"): return str(s);
    else : return "Paid"                   

iCashCol = ['ธนาคาร (iCash)_BBL-1780', 'ธนาคาร (iCash)_BBL-3222','ธนาคาร (iCash)_BBL-4931',]  

chequeCol = ['ธนาคาร (cheque)_BBL-1780','ธนาคาร (cheque)_BBL-3222', 'ธนาคาร (cheque)_BBL-4931','ธนาคาร (cheque)_SCB-3946', 'ธนาคาร (cheque)_TMB-2801','ธนาคาร (cheque)_KTB-0408', 'ธนาคาร (cheque)_BAY-0426','ธนาคาร (cheque)_KBANK-3343', 'ธนาคาร (cheque)_KBANK-1706','ธนาคาร (cheque)_OTHER']


new_csv = pd.DataFrame(columns=new_col)

# จ่ายให้
new_csv[new_col[0]] = df[new_col[0]]

# รายละเอียด
new_csv[new_col[1]] = df[new_col[1]]

# จำนวนเงินรวมทั้งสิ้น
new_csv[new_col[2]] = df[new_col[2]].apply(float)

# เลขที่ cheque
new_csv[new_col[3]] = df['ประเภทการจ่าย']
for i in range(0,len(new_csv[new_col[3]])):
    if(new_csv[new_col[3]].iloc[i] == 'Cheque'):
        new_csv[new_col[3]].iloc[i] = str(df['เลขที่ cheque'].iloc[i]).split('.')[0]
    elif(new_csv[new_col[3]].iloc[i] == 'other'):
        new_csv[new_col[3]].iloc[i] = str(df['ช่องทางอื่นๆ'].iloc[i])
    elif(isBlank(str(new_csv[new_col[3]].iloc[i])) or str(new_csv[new_col[3]].iloc[i]) == "nan"):
        new_csv[new_col[3]].iloc[i] = "NaN" 

# ลงวันที่จ่าย
new_csv[new_col[4]] = df['ลงวันที่จ่าย']      

# A/C NO.
new_csv[new_col[5]] = 'NaN'
for i in range(0,len(new_csv[new_col[3]])):
    if(new_csv[new_col[3]].iloc[i] == 'Cheque'):
        for idx in range(0,len(df[chequeCol].iloc[i])):
            if df[chequeCol].iloc[i][idx] > 0:
                new_csv[new_col[5]].iloc[i] = df[chequeCol].columns[idx]
                # print(df[chequeCol].columns[idx])
                break;    
    elif(new_csv[new_col[3]].iloc[i] == 'iCash'):
        for idx in range(0,len(df[iCashCol].iloc[i])):
            if df[iCashCol].iloc[i][idx] > 0:
                new_csv[new_col[5]].iloc[i] = df[iCashCol].columns[idx]
                # print(df[chequeCol].columns[idx])
                break;   
    elif(new_csv[new_col[3]].iloc[i] == 'other'):
        new_csv[new_col[5]].iloc[i] = 'NaN'   

# A/C NO.
new_csv[new_col[7]] = df['PV Type | PV no. | AP | Paid status'].apply(formatAP)

# task status
new_csv[new_col[8]] = df['task_status'].apply(fillPaid)


In [13]:
# export report

waitPayment = df.loc[df['task_status'].apply(waitpay)].sort_values(by='ลงวันที่จ่าย', ascending=False)


# waitPayment
waitPayment.to_csv(wwwpath+'filter_waitPayment_'+latestfile)

# all data
# last row = summary --> drop last row
new_csv.iloc[:-1].sort_values(by='status', ascending=False).to_csv(wwwpath+'alldata_'+latestfile)

In [14]:
a= "Kamolrat Mindo,Narisara Niyomrat,Nipa Wungsoh"
print(a.split(","))

['Kamolrat Mindo', 'Narisara Niyomrat', 'Nipa Wungsoh']
